<a href="https://colab.research.google.com/github/niksisons/image_processing/blob/main/%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%964_%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B4%D0%B8%D0%BD%D0%B0%D0%BC%D0%B8%D0%BA%D0%B8_%D0%B8%D0%B7%D0%BC%D0%B5%D0%BD%D0%B5%D0%BD%D0%B8%D0%B9_%D0%B7%D0%B5%D0%BC%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BF%D0%BE%D0%BA%D1%80%D0%BE%D0%B2%D0%B0_%D0%BD%D0%B0_%D0%BF%D0%BE%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D1%80%D0%B0%D0%B7%D0%BD%D0%BE%D0%B2%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D1%85_%D1%81%D0%BD%D0%B8%D0%BC%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Проектная работа №4. Анализ динамики изменений земного покрова на последовательности разновременных спутниковых смимков с использованием спектральных индексов**

## **1. Цель работы**

Выполнить количественный анализ динамики изменения классов земного покрова на выбранной территории за период с 2017 по 2024 год на основе разновременных спутниковых снимков Sentinel-2 и провести сравнительную оценку результатов, полученных методом пороговой сегментации на основе спектральных индексов и с использованием готовых карт классификации Dynamic World.

## **2. Задачи работы**

1.  **Подготовить** последовательность разновременных снимков (снимки Sentinel-2 и карты Dynamic World) за 2017-2024 гг.;
2.  **Классифицировать** земной покров на снимках Sentinel-2, применив метод пороговой сегментации по спектральным индексам;
3.  **Рассчитать и сравнить** количественные показатели динамики (площади, темпы роста) для двух методов классификации: по индексам и по данным Dynamic World;
4.  **Оценить** выявленные тренды изменения покрова и **сформулировать** выводы о сопоставимости результатов двух методов анализа.

## **3. Загрузка данных для анализа из Google Earth Engine**

> **Примечание.** В качестве примера реализации можно использовать код из лекционных материалов, доступный по ссылке: https://u.to/nWdJIg. Для выполнения текущей работы актуальны только **Раздел 1** и **Раздел 2** указанного источника.





### **3.1. Настройка рабочей среды и аутентификация**

In [1]:
%pip install -U geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.2 MB/s eta 0:00:00
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 1.5.24
    Uninstalling earthengine-api-1.5.24:
      Successfully uninstalled earthengine-api-1.5.24
  Attempting uninstall: geemap
    Found existing installation: geemap 0.35.3
    Uninstalling geemap-0.35.3:
      Successfully uninstalled geemap-0.35.3


In [2]:
# Установка библиотеки geetools для прямого скачивания
!pip install geetools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.9 MB/s eta 0:00:00


In [3]:
import ee
import geemap

# Укажите ваш Google Cloud Project ID здесь.
# Вы можете найти его в консоли Google Cloud: https://console.cloud.google.com/home/dashboard
PROJECT_ID = 'ee-rogozhinnickita'

# Инициализация Earth Engine
try:
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print("Google Earth Engine initialized successfully!")
except Exception as e:
    print(f"Error initializing Earth Engine: {e}")
    print("Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print("Google Earth Engine authenticated and initialized successfully!")

# Создание папки для экспорта в рамках сессионного хранилища
import os
export_folder_name = 'GEE_Export_Project4'
export_folder_path = os.path.join('/content', export_folder_name)
if not os.path.exists(export_folder_path):
    os.makedirs(export_folder_path)
print(f"Export folder created at: {export_folder_path}")

Google Earth Engine initialized successfully!
Export folder created at: /content/GEE_Export_Project4


### **3.2. Определение области интереса**

- Выберите территорию размером не менее 10×10 км (100 км²) с разнообразным земным покровом
- Обязательно должны присутствовать: водные объекты, растительность, застройка, открытая почва
- Зафиксируйте координаты углов ROI для воспроизводимости результатов

In [22]:
# Определение области интереса (ROI)
# Изменяем ROI на область в Германии (например, к юго-востоку от Берлина)
# Координаты: [min_lon, min_lat, max_lon, max_lat]
min_lon, min_lat = 13.5, 52.2 # Нижний левый угол
max_lon, max_lat = 13.9, 52.5 # Верхний правый угол

roi = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

# Вывод ROI для проверки
print(f"ROI defined: {roi.getInfo()}")

# Вычисление площади ROI
# Площадь возвращается в квадратных метрах, переведем в км2 и га
roi_area_sq_m = roi.area()
roi_area_sq_km = roi_area_sq_m.divide(1e6) # 1e6 = 1,000,000 кв. м в 1 кв. км
roi_area_ha = roi_area_sq_m.divide(10000) # 10,000 кв. м в 1 га

print(f"Площадь ROI: {roi_area_sq_km.getInfo():.2f} км² ({roi_area_ha.getInfo():.2f} га)")

# Визуализация ROI на карте
Map = geemap.Map(center=[(min_lat + max_lat) / 2, (min_lon + max_lon) / 2], zoom=9)
Map.addLayer(roi, {}, 'Область интереса (ROI)')
Map.centerObject(roi)
Map

ROI defined: {'type': 'Polygon', 'coordinates': [[[13.5, 52.2], [13.9, 52.2], [13.9, 52.5], [13.5, 52.5], [13.5, 52.2]]]}
Площадь ROI: 906.31 км² (90630.63 га)


Map(center=[52.35, 13.7], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright',…

In [30]:
# Выполните этот код в новой ячейке или после инициализации GEE
import ee

# Убедитесь, что ROI определена (ячейка 726d2a38 должна быть выполнена)
# min_lon, min_lat = 13.5, 52.2
# max_lon, max_lat = 13.9, 52.5
# roi = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

test_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                    .filterBounds(roi) \
                    .filterDate('2017-01-01', '2020-12-31') \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) # Максимально широкий фильтр

print(f"Количество изображений за 2020 год в ROI: {test_collection.size().getInfo()}")

# Попробуем получить первое изображение, если есть
if test_collection.size().getInfo() > 0:
    first_image = test_collection.first()
    print(f"Первое изображение: {first_image.id().getInfo()}")
else:
    print("Тестовая коллекция все еще пуста. Проблема может быть в Project ID или аутентификации.")

Количество изображений за 2020 год в ROI: 658
Первое изображение: 20170127T102301_20170127T102258_T33UUU


### **3.3. Функции для загрузки и обработки данных Sentinel-2**

In [39]:
def load_sentinel2_data(roi, start_date="2017-01-01", end_date="2020-01-01", cloud_filter=20):
    """
    Загружает и предварительно обрабатывает снимки Sentinel-2.

    Args:
        roi: Область интереса (ee.Geometry).
        start_date (str): Начальная дата в формате 'YYYY-MM-DD'.
        end_date (str): Конечная дата в формате 'YYYY-MM-DD'.
        cloud_filter (int): Максимальный процент облачности (0-100).

    Returns:
        ee.ImageCollection: Отфильтрованная коллекция снимков Sentinel-2.
    """
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate("2017-01-01", "2020-01-01") \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_filter))

    # Определяем основные полосы, которые должны быть в изображении (B2, B3, B4, B8)
    # Это минимальный набор, чтобы изображение считалось валидным для начала обработки.
    initial_bands_check = ee.List(['B2', 'B3', 'B4', 'B8'])
    collection = collection.filter(ee.Filter.notNull(initial_bands_check))

    def mask_and_scale(image):
        # Создаем маску облаков на стороне сервера, используя ee.Algorithms.If
        # Проверяем наличие QA60_BAND_PRESENT на стороне сервера
        qa_band_present = image.bandNames().contains('QA60')

        # Масштабируем все полосы, кроме QA60, если оно есть. Это server-side операция.
        # Сначала масштабируем все доступные полосы
        scaled_image = image.divide(10000)

        # Условное применение маски QA60
        # Если QA60_BAND_PRESENT, используем QA60 для маски, иначе маска не применяется.
        masked_scaled_image = ee.Algorithms.If(
            qa_band_present,
            scaled_image.updateMask(image.select('QA60').bitwiseAnd(1 << 10).eq(0).And(
                                    image.select('QA60').bitwiseAnd(1 << 11).eq(0))),
            scaled_image
        )

        return ee.Image(masked_scaled_image)

    processed_collection = collection.map(mask_and_scale)

    # Определяем все желаемые выходные полосы, включая те, которые могут быть необязательными.
    bands_to_output = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']

    # Функция для добавления недостающих полос с константным значением 0
    # Полностью серверная реализация
    def add_missing_bands_server_side(image):
        # Создаем 'пустое' изображение со всеми требуемыми полосами, заполненными нулями.
        all_zeros = ee.Image([ee.Image.constant(0).rename(b) for b in bands_to_output])

        # Обновляем это изображение значениями из оригинального изображения
        # Это автоматически пропускает полосы, которых нет в оригинале
        image_with_all_bands = all_zeros.addBands(image, bands_to_output, True)

        return image_with_all_bands.select(bands_to_output)

    processed_collection_final = processed_collection.map(add_missing_bands_server_side)

    return processed_collection_final

### **3.4. Создание медианных композитов за летние периоды 2019-2024**

- Используйте медиану для минимизации влияния облачности
- Фильтрация по облачности: CLOUDY_PIXEL_PERCENTAGE < 20%


In [40]:
years = range(2015, 2025) # От 2017 до 2024 года
sentinel2_composites = {}

for year in years:
    # Расширяем временной диапазон (май-сентябрь) и увеличиваем cloud_filter
    start_date = '2020-05-01'
    end_date = '2017-01-01'
    print(f"Processing Sentinel-2 for summer {year} (dates: {start_date} to {end_date})")

    # Загрузка отфильтрованной коллекции Sentinel-2 с увеличенным cloud_filter
    s2_collection = load_sentinel2_data(roi, cloud_filter=100) # Увеличиваем cloud_filter до 80%
    print(f"Количество изображений за 2020 год в ROI: {s2_collection.size().getInfo()}")
    # Создание медианного композита
    if s2_collection.size().getInfo() > 0:
        median_composite = s2_collection.median().clip(roi)
        sentinel2_composites[year] = median_composite
        print(f"  Median composite created for {year}")
    else:
        print(f"  No suitable Sentinel-2 images found for {year} in the specified period/cloud filter.")
        sentinel2_composites[year] = None

print(f"Создано {len([c for c in sentinel2_composites.values() if c is not None])} медианных композитов Sentinel-2.")

# Пример визуализации для 2024 года
if 2024 in sentinel2_composites or sentinel2_composites[2024] is not None:
    sentinel2_vis_params = {
        'min': 0.0,
        'max': 0.3,
        'bands': ['B4', 'B3', 'B2'],
    }
    Map_s2 = geemap.Map(center=[(min_lat + max_lat) / 2, (min_lon + max_lon) / 2], zoom=9)
    Map_s2.addLayer(sentinel2_composites[2024], sentinel2_vis_params, 'Sentinel-2 Summer 2024 (RGB)')
    Map_s2.centerObject(roi)
    Map_s2
else:
    print("Композит Sentinel-2 для 2024 года недоступен для визуализации.")

Processing Sentinel-2 for summer 2015 (dates: 2020-05-01 to 2017-01-01)
Количество изображений за 2020 год в ROI: 0
  No suitable Sentinel-2 images found for 2015 in the specified period/cloud filter.
Processing Sentinel-2 for summer 2016 (dates: 2020-05-01 to 2017-01-01)
Количество изображений за 2020 год в ROI: 0
  No suitable Sentinel-2 images found for 2016 in the specified period/cloud filter.
Processing Sentinel-2 for summer 2017 (dates: 2020-05-01 to 2017-01-01)
Количество изображений за 2020 год в ROI: 0
  No suitable Sentinel-2 images found for 2017 in the specified period/cloud filter.
Processing Sentinel-2 for summer 2018 (dates: 2020-05-01 to 2017-01-01)
Количество изображений за 2020 год в ROI: 0
  No suitable Sentinel-2 images found for 2018 in the specified period/cloud filter.
Processing Sentinel-2 for summer 2019 (dates: 2020-05-01 to 2017-01-01)
Количество изображений за 2020 год в ROI: 0
  No suitable Sentinel-2 images found for 2019 in the specified period/cloud fil

### **3.5. Функции для загрузки и обработки данных карт классификации**

**Используйте коллекцию Dynamic World (Google/WRI)**

**Коллекция в GEE:** `ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")`
- **Период:** с июня 2015 года по настоящее время (обновляется постоянно)
- **Разрешение:** 10 метров
- **Особенности:**
  - Предоставляет вероятности для каждого класса (не жесткую классификацию)
  - Обновляется почти в реальном времени для каждого снимка Sentinel-2
  - 9 классов земного покрова


```python
# Пример загрузки Dynamic World

def load_landcover_data(roi, start_date, end_date, composite='mode'):
    """
    Загружает карты классификации Google Dynamic World (10м).
    
    Классы: 0-Вода, 1-Деревья, 2-Трава, 3-Затопл.растительность,
            4-Культуры, 5-Кустарники, 6-Застройка, 7-Голая земля, 8-Снег/лед
    
    Args:
        roi: область интереса
        start_date, end_date: период 'YYYY-MM-DD'
        composite: 'mode'|'first'|'mosaic'|'collection'
    """
    
    labels = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1") \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .select('label')
    
    if composite == 'collection':
        return labels.map(lambda img: img.clip(roi))
    
    return getattr(labels, composite)().clip(roi)

# Загрузка с композитом по моде (наиболее частый класс)
landcover = load_landcover_data(roi, '2024-06-01', '2024-08-31')

# Параметры визуализации для Dynamic World
landcover_vis = {
    'min': 0,
    'max': 8,
    'palette': [
        '419bdf',  # 0 - Вода - синий
        '397d49',  # 1 - Деревья - темно-зеленый  
        '88b053',  # 2 - Трава - светло-зеленый
        '7a87c6',  # 3 - Затопленная растительность - сине-фиолетовый
        'e49635',  # 4 - Культуры - оранжево-желтый
        'dfc35a',  # 5 - Кустарники - песочный
        'c4281b',  # 6 - Застройка - красный
        'a59b8f',  # 7 - Голая земля - серо-коричневый
        'b39fe1'   # 8 - Снег и лед - светло-фиолетовый (для контраста)
    ]
}

# Визуализация на карте
map_dw = geemap.Map()
map_dw.centerObject(roi, 11)
map_dw.addLayer(landcover, landcover_vis, 'Dynamic World')
map_dw

```

In [12]:
def load_landcover_data(roi, start_date, end_date, composite='mode'):
    """
    Загружает карты классификации Google Dynamic World (10м).

    Классы: 0-Вода, 1-Деревья, 2-Трава, 3-Затопл.растительность,
            4-Культуры, 5-Кустарники, 6-Застройка, 7-Голая земля, 8-Снег/лед

    Args:
        roi: область интереса
        start_date, end_date: период 'YYYY-MM-DD'
        composite: 'mode'|'first'|'mosaic'|'collection'
    """

    labels = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1") \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .select('label')

    if composite == 'collection':
        return labels.map(lambda img: img.clip(roi))

    return getattr(labels, composite)().clip(roi)

# Загрузка с композитом по моде (наиболее частый класс)
landcover = load_landcover_data(roi, '2024-06-01', '2024-08-31')

# Параметры визуализации для Dynamic World
landcover_vis = {
    'min': 0,
    'max': 8,
    'palette': [
        '419bdf',  # 0 - Вода - синий
        '397d49',  # 1 - Деревья - темно-зеленый
        '88b053',  # 2 - Трава - светло-зеленый
        '7a87c6',  # 3 - Затопленная растительность - сине-фиолетовый
        'e49635',  # 4 - Культуры - оранжево-желтый
        'dfc35a',  # 5 - Кустарники - песочный
        'c4281b',  # 6 - Застройка - красный
        'a59b8f',  # 7 - Голая земля - серо-коричневый
        'b39fe1'   # 8 - Снег и лед - светло-фиолетовый (для контраста)
    ]
}

# Визуализация на карте
map_dw = geemap.Map()
map_dw.centerObject(roi, 11)
map_dw.addLayer(landcover, landcover_vis, 'Dynamic World')
map_dw

Map(center=[52.34999914811299, 13.699999999999187], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
def load_landcover_data(roi, start_date, end_date, composite='mode'):
    """
    Загружает карты классификации Google Dynamic World (10м).

    Классы: 0-Вода, 1-Деревья, 2-Трава, 3-Затопл.растительность,
            4-Культуры, 5-Кустарники, 6-Застройка, 7-Голая земля, 8-Снег/лед

    Args:
        roi: область интереса
        start_date, end_date: период 'YYYY-MM-DD'
        composite: 'mode'|'first'|'mosaic'|'collection'
    """

    labels = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1") \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .select('label')

    if composite == 'collection':
        return labels.map(lambda img: img.clip(roi))

    # Используем mode() для получения наиболее частого класса за период
    # Если коллекция пуста, tobeArray() выдаст ошибку, поэтому проверяем размер.
    if labels.size().getInfo() > 0:
        return getattr(labels, composite)().clip(roi)
    else:
        return None

### **3.4. Создание медианных карт классификации за летние периоды 2019-2024**

In [ ]:
dynamic_world_composites = {}

# Параметры визуализации для Dynamic World
landcover_vis_params = {
    'min': 0,
    'max': 8,
    'palette': [
        '419bdf',  # 0 - Вода - синий
        '397d49',  # 1 - Деревья - темно-зеленый
        '88b053',  # 2 - Трава - светло-зеленый
        '7a87c6',  # 3 - Затопленная растительность - сине-фиолетовый
        'e49635',  # 4 - Культуры - оранжево-желтый
        'dfc35a',  # 5 - Кустарники - песочный
        'c4281b',  # 6 - Застройка - красный
        'a59b8f',  # 7 - Голая земля - серо-коричневый
        'b39fe1'   # 8 - Снег и лед - светло-фиолетовый (для контраста)
    ]
}

for year in years:
    start_date = f'{year}-06-01'
    end_date = f'{year}-08-31'
    print(f"Processing Dynamic World for summer {year} (dates: {start_date} to {end_date})")

    # Загрузка и создание композита Dynamic World (используем 'mode' для получения наиболее частого класса)
    landcover_composite = load_landcover_data(roi, start_date, end_date, composite='mode')

    if landcover_composite is not None:
        dynamic_world_composites[year] = landcover_composite
        print(f"  Dynamic World composite created for {year}")
    else:
        print(f"  No suitable Dynamic World images found for {year} in the specified period.")
        dynamic_world_composites[year] = None

print(f"Создано {len([c for c in dynamic_world_composites.values() if c is not None])} медианных композитов Dynamic World.")

# Пример визуализации для 2024 года
if 2024 in dynamic_world_composites and dynamic_world_composites[2024] is not None:
    Map_dw = geemap.Map(center=[(min_lat + max_lat) / 2, (min_lon + max_lon) / 2], zoom=9)
    Map_dw.addLayer(dynamic_world_composites[2024], landcover_vis_params, 'Dynamic World Summer 2024')
    Map_dw.centerObject(roi)
    Map_dw
else:
    print("Композит Dynamic World для 2024 года недоступен для визуализации.")

### **3.5. Экспорт полученных растров**

- **Медианные композиты** Sentinel-2 (7 файлов GeoTIFF):
  - `sentinel2_summer_2017.tif` (июнь-август 2017)
  - `sentinel2_summer_2018.tif` (июнь-август 2018)
  - `sentinel2_summer_2019.tif` (июнь-август 2019)
  - `sentinel2_summer_2020.tif` (июнь-август 2020)
  - `sentinel2_summer_2021.tif` (июнь-август 2021)
  - `sentinel2_summer_2022.tif` (июнь-август 2022)
  - `sentinel2_summer_2023.tif` (июнь-август 2023)
  - `sentinel2_summer_2024.tif` (июнь-август 2024)

- **Медианные карты классификации** (7 файлов GeoTIFF):
  - `DYNAMICWORLD_summer_2017.tif` (июнь-август 2017)
  - `DYNAMICWORLD_summer_2018.tif` (июнь-август 2018)
  - `DYNAMICWORLD_summer_2019.tif` (июнь-август 2019)
  - `DYNAMICWORLD_summer_2020.tif` (июнь-август 2020)
  - `DYNAMICWORLD_summer_2021.tif` (июнь-август 2021)
  - `DYNAMICWORLD_summer_2022.tif` (июнь-август 2022)
  - `DYNAMICWORLD_summer_2023.tif` (июнь-август 2023)
  - `DYNAMICWORLD_summer_2024.tif` (июнь-август 2024)

In [ ]:
import os
from geetools.batch import download # Импортируем модуль download из geetools.batch

# Определим папку для экспорта, созданную ранее
EXPORT_FOLDER = export_folder_path # '/content/GEE_Export_Project4'

# Разрешение для экспорта (Sentinel-2 имеет 10м)
export_scale = 10

print(f"Начинается экспорт GeoTIFF файлов в папку: {EXPORT_FOLDER}")

sentinel2_files = []
# --- Экспорт медианных композитов Sentinel-2 ---
for year, composite in sentinel2_composites.items():
    if composite is not None:
        file_name = f'sentinel2_summer_{year}.tif'
        file_path = os.path.join(EXPORT_FOLDER, file_name)
        sentinel2_files.append(file_path)

        print(f"  Загрузка Sentinel-2 для {year} в {file_path}...")
        try:
            download.to_local(
                collection=ee.ImageCollection([composite.float()]), # to_local ожидает ImageCollection
                folder=EXPORT_FOLDER,
                name=f'sentinel2_summer_{year}', # Имя файла без расширения
                scale=export_scale,
                region=roi.getInfo(),
                crs='EPSG:4326', # Указываем CRS
                max_pixels=1e13,
                file_per_band=False # Скачать все каналы в один файл
            )
            print(f"  Sentinel-2 для {year} успешно загружен.")
        except Exception as e:
            print(f"  Ошибка при загрузке Sentinel-2 для {year}: {e}")
    else:
        print(f"  Пропущен экспорт Sentinel-2 для {year} (композит недоступен).")

dynamic_world_files = []
# --- Экспорт медианных карт классификации Dynamic World ---
for year, composite_dw in dynamic_world_composites.items():
    if composite_dw is not None:
        file_name = f'DYNAMICWORLD_summer_{year}.tif'
        file_path = os.path.join(EXPORT_FOLDER, file_name)
        dynamic_world_files.append(file_path)

        print(f"  Загрузка Dynamic World для {year} в {file_path}...")
        try:
            download.to_local(
                collection=ee.ImageCollection([composite_dw.select('label').unmask(-1).int()]), # Оборачиваем в ImageCollection
                folder=EXPORT_FOLDER,
                name=f'DYNAMICWORLD_summer_{year}', # Имя файла без расширения
                scale=export_scale,
                region=roi.getInfo(),
                crs='EPSG:4326', # Указываем CRS
                max_pixels=1e13,
                file_per_band=False
            )
            print(f"  Dynamic World для {year} успешно загружен.")
        except Exception as e:
            print(f"  Ошибка при загрузке Dynamic World для {year}: {e}")
    else:
        print(f"  Пропущен экспорт Dynamic World для {year} (композит недоступен).")

print("Все задачи по загрузке GeoTIFF файлов завершены.")

## **4. Работа с загруженными растрами в RasterIO**

### **4.1. Загрузка и проверка растров**

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os

# Путь к папке с экспортированными файлами
# Убедимся, что export_folder_path определен, если ячейки запускаются не по порядку.
# Если 'export_folder_path' не определен, определим его здесь.
try:
    EXPORT_FOLDER = export_folder_path
except NameError:
    export_folder_name = 'GEE_Export_Project4'
    EXPORT_FOLDER = os.path.join('/content', export_folder_name)
    print(f"Переменная 'export_folder_path' не была определена. Используется '{EXPORT_FOLDER}'.")

# Список файлов Sentinel-2 для загрузки
sentinel2_exported_files = []
# Нужно определить 'years' и 'sentinel2_composites' если они не определены
# Для надежности, можно перезапустить ячейки, где они определяются, или передать их.
# Однако, в контексте этого блокнота, они должны быть определены в предыдущих ячейках.
for year in range(2017, 2025): # Примерный диапазон лет, должен совпадать с 'years' из предыдущих ячеек
    # Проверяем наличие файла на диске, так как composite мог быть None
    file_path = os.path.join(EXPORT_FOLDER, f'sentinel2_summer_{year}.tif')
    if os.path.exists(file_path):
        sentinel2_exported_files.append(file_path)

print(f"Найдено {len(sentinel2_exported_files)} файлов Sentinel-2 для визуализации.")

# Визуализация всех растров в видимом диапазоне (B4, B3, B2)
fig, axes = plt.subplots(1, len(sentinel2_exported_files), figsize=(20, 10))
fig.suptitle('Sentinel-2 Composites (RGB) for Summer Periods', fontsize=16)

# Обработка случая, когда axes не является массивом (для одного файла)
if len(sentinel2_exported_files) == 1:
    axes = [axes]

for i, file_path in enumerate(sentinel2_exported_files):
    year = int(os.path.basename(file_path).split('_')[-1].split('.')[0])
    try:
        with rasterio.open(file_path) as src:
            # Читаем каналы B4 (Red), B3 (Green), B2 (Blue)
            # Каналы в Sentinel-2 SR обычно: B2, B3, B4, B8, B11, B12
            # Предполагаем, что порядок в экспорте остался таким же: B2=1, B3=2, B4=3
            blue = src.read(1) # B2
            green = src.read(2) # B3
            red = src.read(3) # B4

            # Нормализация для визуализации
            def normalize(band):
                # Заменяем NaN на 0 перед нормализацией
                band_clean = np.nan_to_num(band, nan=0.0)
                min_val = np.min(band_clean)
                max_val = np.max(band_clean)
                if max_val == min_val:
                    return np.zeros_like(band_clean) # Избегаем деления на ноль
                return (band_clean - min_val) / (max_val - min_val)

            rgb = np.stack([normalize(red), normalize(green), normalize(blue)], axis=-1)

            ax = axes[i]
            ax.imshow(rgb)
            ax.set_title(f'Summer {year}')
            ax.axis('off')
    except Exception as e:
        print(f"Ошибка при чтении или визуализации файла {file_path}: {e}")

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os

# Путь к папке с экспортированными файлами
try:
    EXPORT_FOLDER = export_folder_path
except NameError:
    export_folder_name = 'GEE_Export_Project4'
    EXPORT_FOLDER = os.path.join('/content', export_folder_name)
    print(f"Переменная 'export_folder_path' не была определена. Используется '{EXPORT_FOLDER}'.")

# Список файлов Dynamic World для загрузки
dynamic_world_exported_files = []
# Нужно определить 'years' и 'dynamic_world_composites' если они не определены
# Для надежности, можно перезапустить ячейки, где они определяются, или передать их.
for year in range(2017, 2025): # Примерный диапазон лет, должен совпадать с 'years' из предыдущих ячеек
    # Проверяем наличие файла на диске
    file_path = os.path.join(EXPORT_FOLDER, f'DYNAMICWORLD_summer_{year}.tif')
    if os.path.exists(file_path):
        dynamic_world_exported_files.append(file_path)

print(f"Найдено {len(dynamic_world_exported_files)} файлов Dynamic World для визуализации.")

# Параметры визуализации для Dynamic World (из предыдущей ячейки)
landcover_vis_params = {
    'min': 0,
    'max': 8,
    'palette': [
        '#419bdf',  # 0 - Вода - синий
        '#397d49',  # 1 - Деревья - темно-зеленый
        '#88b053',  # 2 - Трава - светло-зеленый
        '#7a87c6',  # 3 - Затопленная растительность - сине-фиолетовый
        '#e49635',  # 4 - Культуры - оранжево-желтый
        '#dfc35a',  # 5 - Кустарники - песочный
        '#c4281b',  # 6 - Застройка - красный
        '#a59b8f',  # 7 - Голая земля - серо-коричневый
        '#b39fe1'   # 8 - Снег и лед - светло-фиолетовый (для контраста)
    ]
}

# Создание colormap из палитры
from matplotlib.colors import ListedColormap
cmap = ListedColormap([color for color in landcover_vis_params['palette']])

# Визуализация всех растров Dynamic World
fig, axes = plt.subplots(1, len(dynamic_world_exported_files), figsize=(20, 10))
fig.suptitle('Dynamic World Landcover Maps for Summer Periods', fontsize=16)

# Обработка случая, когда axes не является массивом (для одного файла)
if len(dynamic_world_exported_files) == 1:
    axes = [axes]

for i, file_path in enumerate(dynamic_world_exported_files):
    year = int(os.path.basename(file_path).split('_')[-1].split('.')[0])
    try:
        with rasterio.open(file_path) as src:
            # Читаем единственный канал 'label'
            landcover_data = src.read(1)
            # Заменяем -1 (unmask value) на NaN для корректной визуализации или другую метку
            landcover_data = np.where(landcover_data == -1, np.nan, landcover_data)

            ax = axes[i]
            im = ax.imshow(landcover_data, cmap=cmap, vmin=landcover_vis_params['min'], vmax=landcover_vis_params['max'])
            ax.set_title(f'Summer {year}')
            ax.axis('off')
    except Exception as e:
        print(f"Ошибка при чтении или визуализации файла {file_path}: {e}")

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### **4.2. Рассчитать все 8 индексов для каждого года (2015-2024)**


> **Примечание.** В качестве примера рассчета индексов можно использовать код из лекционных материалов, доступный по ссылке: https://u.to/4qBeIg





**Спектральные индексы** — это математические комбинации значений отражения в разных спектральных каналах, позволяющие выделить определенные типы объектов.








#### **Таблица 1. Основные спектральные индексы для анализа земного покрова**




| **Индекс** | **Формула** | **Назначение** | **Каналы Sentinel-2** |
|------------|-------------|----------------|------------------------|
| **NDVI** (Normalized Difference Vegetation Index) | (NIR - RED) / (NIR + RED) | Оценка состояния растительности | (B8 - B4) / (B8 + B4) |
| **NDWI** (Normalized Difference Water Index) | (GREEN - NIR) / (GREEN + NIR) | Выделение водных объектов | (B3 - B8) / (B3 + B8) |
| **NDBI** (Normalized Difference Built-up Index) | (SWIR - NIR) / (SWIR + NIR) | Выделение застроенных территорий | (B11 - B8) / (B11 + B8) |
| **SAVI** (Soil Adjusted Vegetation Index) | ((NIR - RED) / (NIR + RED + L)) * (1 + L), L=0.5 | Растительность с учетом почвы | ((B8 - B4) / (B8 + B4 + 0.5)) * 1.5 |
| **EVI** (Enhanced Vegetation Index) | 2.5 * ((NIR - RED) / (NIR + 6*RED - 7.5*BLUE + 1)) | Улучшенный вегетационный индекс | 2.5 * ((B8 - B4) / (B8 + 6*B4 - 7.5*B2 + 1)) |
| **MNDWI** (Modified NDWI) | (GREEN - SWIR) / (GREEN + SWIR) | Модифицированный водный индекс | (B3 - B11) / (B3 + B11) |
| **BSI** (Bare Soil Index) | ((SWIR + RED) - (NIR + BLUE)) / ((SWIR + RED) + (NIR + BLUE)) | Открытая почва | ((B11 + B4) - (B8 + B2)) / ((B11 + B4) + (B8 + B2)) |
| **NDSI** (Normalized Difference Snow Index) | (GREEN - SWIR) / (GREEN + SWIR) | Снежный покров | (B3 - B11) / (B3 + B11) |


#### **Таблица 2. Интерпретация значений NDVI**



| **Диапазон NDVI** | **Тип покрова** | **Характеристика** |
|-------------------|-----------------|---------------------|
| < 0.0 | Вода | Водные поверхности, снег |
| 0.0 - 0.1 | Открытая почва | Песок, камни, бетон |
| 0.1 - 0.2 | Разреженная растительность | Степи, пустыни |
| 0.2 - 0.4 | Умеренная растительность | Кустарники, луга |
| 0.4 - 0.6 | Плотная растительность | Лиственные леса |
| > 0.6 | Очень плотная растительность | Тропические леса, хвойные леса |

#### **Таблица 3. Интерпретация значений NDWI**



| **Диапазон NDWI** | **Тип объекта** | **Описание** |
|-------------------|-----------------|--------------|
| > 0.3 | Водные объекты | Реки, озера, пруды |
| 0.0 - 0.3 | Влажная почва/растительность | Болота, заливные луга |
| -0.3 - 0.0 | Сухая растительность | Обычная растительность |
| < -0.3 | Застройка/почва | Городская застройка, открытая почва |

#### **Таблица 4. Интерпретация значений NDBI**



| **Диапазон NDBI** | **Тип территории** | **Характеристика** |
|-------------------|-------------------|---------------------|
| > 0.0 | Застроенные территории | Города, промзоны |
| -0.1 - 0.0 | Смешанные территории | Пригороды, села |
| < -0.1 | Природные территории | Леса, поля, водоемы |

#### **Таблица 5. Интерпретация значений SAVI**



| **Диапазон SAVI** | **Тип покрова** | **Описание** |
|-------------------|-----------------|---------------|
| < 0.1 | Открытая почва/вода | Водные объекты, голая почва |
| 0.1 - 0.2 | Очень разреженная растительность | Пустыни, каменистые участки |
| 0.2 - 0.4 | Разреженная растительность | Степи, саванны |
| 0.4 - 0.6 | Умеренная растительность | Сельхозугодья, луга |
| > 0.6 | Плотная растительность | Леса, густые кустарники |

#### **Таблица 6. Интерпретация значений EVI**



| **Диапазон EVI** | **Состояние растительности** | **Характеристика** |
|------------------|------------------------------|---------------------|
| < 0.0 | Нет растительности | Вода, снег, облака |
| 0.0 - 0.2 | Минимальная растительность | Голая почва, урбанизированные территории |
| 0.2 - 0.3 | Низкая биомасса | Травянистая растительность |
| 0.3 - 0.5 | Средняя биомасса | Кустарники, молодые посевы |
| 0.5 - 0.8 | Высокая биомасса | Зрелые леса, интенсивное сельское хозяйство |
| > 0.8 | Очень высокая биомасса | Тропические леса |

#### **Таблица 7. Интерпретация значений MNDWI**



| **Диапазон MNDWI** | **Тип объекта** | **Характеристика** |
|--------------------|-----------------|---------------------|
| > 0.5 | Глубокая вода | Озера, моря, водохранилища |
| 0.2 - 0.5 | Мелкая вода | Реки, каналы, мелководья |
| 0.0 - 0.2 | Влажные территории | Болота, рисовые поля |
| < 0.0 | Суша | Растительность, почва, застройка |

#### **Таблица 8. Интерпретация значений BSI**



| **Диапазон BSI** | **Тип поверхности** | **Описание** |
|-------------------|-------------------|---------------|
| > 0.1 | Открытая почва | Пашни, карьеры, пустыни |
| 0.0 - 0.1 | Частично покрытая почва | Разреженная растительность |
| -0.1 - 0.0 | Смешанный покров | Переходные зоны |
| < -0.1 | Растительность/вода | Леса, водоемы, плотная растительность |

#### **Таблица 9. Интерпретация значений NDSI**



| **Диапазон NDSI** | **Тип покрова** | **Характеристика** |
|-------------------|-----------------|---------------------|
| > 0.4 | Снежный покров | Чистый снег, ледники |
| 0.2 - 0.4 | Частичный снежный покров | Смешанный снег с растительностью |
| 0.0 - 0.2 | Возможный снег/лед | Тающий снег, иней, облака |
| -0.2 - 0.0 | Переходная зона | Влажная почва, туман |
| < -0.2 | Бесснежные территории | Растительность, почва, вода |

In [ ]:
# Ваш код

### **4.3. Визуализировать каждый индекс для 2024 года с соответствующей цветовой шкалой**



In [ ]:
# Ваш код

### **4.4. Сохранить все рассчитанные индексы в отдельные GeoTIFF файлы с сохранением геопривязки (сохранить в соответствующие папки по годам)**



In [ ]:
# Ваш код

## **5. Сегментация и классификация**

**Сегментация** — процесс разделения изображения на однородные регионы (сегменты) на основе заданных критериев .


### **5.1. Многопороговая сегментация с использованием спектральных индексов**



**Задание:** Реализовать классификацию земного покрова путем замены значений пикселей в растрах индексов на метки классов согласно пороговым правилам.

**Пояснение:** По сути необходимо преобразовать непрерывные значения индексов в дискретные классы (метки) на основе пороговых значений из Таблиц 2-9. Каждому пикселю присваивается числовая метка класса (0-8) в зависимости от значений спектральных индексов.






**Алгоритм многопороговой сегментации:**
1. Создать пустой растр для меток классов (заполнить значением -1 или 255 для "неклассифицировано")
2. Последовательно применить правила классификации
3. Каждое правило проверяется только для еще не классифицированных пикселей
4. Присвоить соответствующую метку класса пикселям, удовлетворяющим условию

**Иерархия правил классификации (применять строго последовательно):**
1. **Вода (метка=0)**: (NDWI > 0.3) ИЛИ (MNDWI > 0.2)
2. **Снег/лед (метка=1)**: (NDSI > 0.4) И (еще не классифицировано)
3. **Застройка (метка=2)**: (NDBI > 0.0) И (NDVI < 0.2) И (еще не классифицировано)
4. **Густая растительность (метка=3)**: ((NDVI > 0.6) ИЛИ (EVI > 0.5)) И (еще не классифицировано)
5. **Умеренная растительность (метка=4)**: (0.3 ≤ NDVI ≤ 0.6) И (еще не классифицировано)
6. **Разреженная растительность (метка=5)**: (0.1 ≤ NDVI < 0.3) И (еще не классифицировано)
7. **Открытая почва (метка=6)**: (BSI > 0.0) И (NDVI < 0.1) И (еще не классифицировано)
8. **Прочее (метка=7)**: все оставшиеся неклассифицированные пиксели


**(МОЖЕТЕ СФОРМУЛИРОВАТЬ СВОИ ПРАВИЛА, ОСНОВЫВАЯСЬ НА ТАБЛИЦАХ, ПРИВЕДЕННЫХ ВЫШЕ!)**

In [ ]:
# Ваш код


### **5.2. Подсчет количества пикселей каждого класса**



**Задание:** После замены значений индексов на метки классов, подсчитать количество пикселей для каждой метки класса.

**Пояснение:** Используйте функции numpy.unique() или аналогичные для подсчета количества пикселей с каждой меткой. Это позволит получить статистику распределения классов на изображении.

**Что нужно получить:**
- Массив уникальных меток классов (0-7)
- Количество пикселей для каждой метки
- Процентное соотношение классов от общего числа пикселей

In [ ]:
# Ваш код

### **5.3. Применение сегментации ко всем временным срезам**




**Задание:** Применить разработанный алгоритм сегментации ко всем медианным композитам (2017-2024).

**Пояснение:** Использовать одинаковые пороговые значения для всех лет для обеспечения сопоставимости результатов при анализе динамики. Сохранить результаты сегментации как растры с метками классов.

**Выходные данные:**
- 8 растров с метками классов (по одному на каждый год)
- Таблица с количеством пикселей каждого класса для каждого года
- Формат сохранения: GeoTIFF с сохранением геопривязки

In [ ]:
# Ваш код

### **5.4. Визуализация результатов сегментации**






**Задание:** Создать тематические карты классификации с легендой.

**Пояснение:** Назначить каждой метке класса соответствующий цвет для визуализации. Использовать стандартную цветовую схему для классов земного покрова.

**Цветовая схема для визуализации:**
- 0 - Вода: синий (#419bdf)
- 1 - Снег/лед: белый (#ffffff)  
- 2 - Застройка: красный (#c4281b)
- 3 - Густая растительность: темно-зеленый (#397d49)
- 4 - Умеренная растительность: зеленый (#88b053)
- 5 - Разреженная растительность: светло-зеленый (#7a87c6)
- 6 - Открытая почва: коричневый (#a59b8f)
- 7 - Прочее: серый (#808080)

In [ ]:
# Ваш код

## **6. Анализ динамики изменений на сегментированных растрах**

### **6.1. Рассчитать площади всех классов для каждого года**



**Формула перевода пикселей в площадь:**
```
Площадь (га) = Количество_пикселей × (Разрешение_м)² / 10000
Площадь (км²) = Количество_пикселей × (Разрешение_м)² / 1000000
```
Для Sentinel-2: разрешение = 10 м, следовательно:
- 1 пиксель = 100 м² = 0.01 га
- 100 пикселей = 1 га
- 10000 пикселей = 1 км²

In [ ]:
# Ваш код

### **6.2. Построить графики динамики изменений для каждого класса**



In [ ]:
# Ваш код

### **6.3. Рассчитать все статистические показатели**



**Обязательные метрики для каждого класса:**

1. **Абсолютные показатели:**
   - Площадь в га и км² для каждого года
   - Количество пикселей
   - Процент от общей площади ROI

2. **Показатели изменений:**
   - Абсолютное изменение площади (га): ΔS = S₂₀₂₄ - S₂₀₁₇
   - Относительное изменение (%): (S₂₀₂₄ - S₂₀₁₇) / S₂₀₁₇ × 100%
   - Среднегодовой прирост (га/год): ΔS / количество_лет
   - Среднегодовой темп роста (%/год): ((S₂₀₂₄/S₂₀₁₇)^(1/7) - 1) × 100%

3. **Статистика временного ряда:**
   - Коэффициент вариации: CV = σ/μ × 100% (вспомните материал прошлого семестра, а именно работу в numpy)
   - Стандартное отклонение площадей

In [ ]:
# Ваш код

### **6.4. Определить какие классы показали наибольшие изменения**



**Критерии определения значимых изменений:**
- Изменение площади > 10% от начального значения
- Абсолютное изменение > 10 га
- Ранжировать классы по величине относительных изменений

In [ ]:
# Ваш код

### **6.5. Создать сводную таблицу результатов**



| Класс | S_2017 (га) | S_2024 (га) | ΔS (га) | Δ% | Темп (%/год) | Тренд | CV (%) |
|-------|-------------|-------------|---------|-----|--------------|-------|-----|
| Вода | ... | ... | ... | ... | ... | ↑/↓/→ | ... |
| Растительность | ... | ... | ... | ... | ... | ↑/↓/→ | ... |
| ... | ... | ... | ... | ... | ... | ... | ... |

In [ ]:
# Ваш код

## **7. Валидация результатов**

### **7.1. Рассчитать площади всех классов для каждого года по картам классификации**



In [ ]:
# Ваш код

### **7.2. Построить графики динамики изменений для каждого класса по картам классификации**



In [ ]:
# Ваш код

### **7.3. Рассчитать все статистические показатели по картам классификации**



In [ ]:
# Ваш код

### **7.4. Определить какие классы показали наибольшие изменения по картам классификации**



In [ ]:
# Ваш код

### **7.5. Создать сводную таблицу результатов по картам классификации**



In [ ]:
# Ваш код

## **8. Сравнение результатов полученных по результатам сегментации и картам классификаций**

In [ ]:
# Ваш код

## **9. Выводы**

В выводах обязательно отразить:
1. **Количественная оценка динамики** (с конкретными цифрами для каждого класса)
2. **Выявленные тренды** (растущие, убывающие, стабильные классы)
3. **Скорость изменений** (га/год и %/год для ключевых классов)
4. **Пространственные закономерности** (где происходят основные изменения)
5. **Точность методов** (сравнение с эталоном в %)



```
# ВАШИ ВЫВОДЫ
```

